<a href="https://colab.research.google.com/github/IvanDePivan/2AMM30-groep-2-component-1/blob/main/flair_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import random

def write(filename, text):
  with open(f"{filename}.conll", "w") as file1:
    file1.write("\n\n".join(text))

with open('annotated_trim.conll') as f:
  data = f.read()
  splits = data.split(2*os.linesep)
  print(len(splits))
  splits = sorted(splits, key=lambda x: random.random())
  train = splits[:58]
  test = splits[58:68]
  dev = splits[68:78]
  print(len(train))
  print(len(test))
  print(len(dev))

  write('train', train)
  write('test', test)
  write('dev', dev)


78
58
10
10


In [ ]:
import torch

torch.cuda.is_available()

True

In [ ]:
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 59.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 50.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB

In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

columns = {0: 'text', 1: 'ner'}
data_folder = ''

# corpus: Corpus = ColumnCorpus(data_folder, columns,
#                               train_file='annotated_trim.conll',
#                               test_file='annotated_trim.conll',
#                               dev_file='annotated_trim.conll')

corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.conll',
                              test_file='test.conll',
                              dev_file='dev.conll')
print(f'Corpus size: {len(corpus.train)}')
print(corpus.test[0].to_tagged_string('ner'))

2023-10-11 11:28:25,160 Reading data from .
2023-10-11 11:28:25,161 Train: train.conll
2023-10-11 11:28:25,164 Dev: dev.conll
2023-10-11 11:28:25,165 Test: test.conll
Corpus size: 57
Sentence[284]: "Michel Gustave Édouard Mayor ( born 12 January 1942 ) is a Swiss astrophysicist and professor emeritus at the University of Geneva 's Department of Astronomy. He formally retired in 2007 , but remains active as a researcher at the Observatory of Geneva. He is co-laureate of the 2019 Nobel Prize in Physics along with Jim Peebles and Didier Queloz , and the winner of the 2010 Viktor Ambartsumian International Prize and the 2015 Kyoto Prize. Together with Didier Queloz in 1995 , he discovered , the first extrasolar planet orbiting a sun-like star , 51 Pegasi. For this achievement , they were awarded the 2019 Nobel Prize in Physics " for the discovery of an exoplanet orbiting a solar-type star " resulting in " contributions to our understanding of the evolution of the universe and Earth 's plac

In [ ]:
# tag to predict
tag_type = 'ner'# make tag dictionary from the corpus
tag_dictionary = corpus.make_label_dictionary(label_type=tag_type)

2023-10-11 11:28:25,584 Computing label dictionary. Progress:


57it [00:00, 13005.95it/s]

2023-10-11 11:28:25,595 Dictionary created for label 'ner' with 6 values: WINNER (seen 555 times), DATE (seen 376 times), NATIONALITY (seen 265 times), PRIZETYPE (seen 56 times), REASON (seen 36 times)


In [ ]:
from flair.embeddings import WordEmbeddings, StackedEmbeddings, TokenEmbeddings
from typing import List

embedding_types : List[TokenEmbeddings] = [
        WordEmbeddings('glove'),
        ## other embeddings
        ]

embeddings : StackedEmbeddings = StackedEmbeddings(
                                 embeddings=embedding_types)

In [ ]:
from flair.models import SequenceTagger
tagger : SequenceTagger = SequenceTagger(hidden_size=256,
                                       embeddings=embeddings,
                                       tag_dictionary=tag_dictionary,
                                       tag_type=tag_type,
                                       use_crf=True,
                                         tag_format='BIO'
                                         )
print(tagger)

2023-10-11 11:28:29,345 SequenceTagger predicts: Dictionary with 21 tags: O, S-WINNER, B-WINNER, E-WINNER, I-WINNER, S-DATE, B-DATE, E-DATE, I-DATE, S-NATIONALITY, B-NATIONALITY, E-NATIONALITY, I-NATIONALITY, S-PRIZETYPE, B-PRIZETYPE, E-PRIZETYPE, I-PRIZETYPE, S-REASON, B-REASON, E-REASON, I-REASON
SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=23, bias=True)
  (loss_function): ViterbiLoss()
  (crf): CRF()
)


In [ ]:
from flair.trainers import ModelTrainer
trainer : ModelTrainer = ModelTrainer(tagger, corpus)
trainer.train('resources/taggers/ner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=300) # 150



2023-10-11 11:28:29,649 ----------------------------------------------------------------------------------------------------
2023-10-11 11:28:29,651 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=23, bias=True)
  (loss_function): ViterbiLoss()
  (crf): CRF()
)"
2023-10-11 11:28:29,652 ----------------------------------------------------------------------------------------------------
2023-10-11 11:28:29,654 Corpus: "Corpus: 57 train + 10 dev + 10 test sentences"
2023-10-11 11:28:29,655 ----------------------------------------------------------------------------------------------------
2023-10-11 11:28:29,657 Param

/usr/local/lib/python3.10/dist-packages/flair/trainers/trainer.py:69: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  warnings.warn(


2023-10-11 11:28:30,740 epoch 1 - iter 1/2 - loss 3.69531250 - time (sec): 1.06 - samples/sec: 9422.71 - lr: 0.100000
2023-10-11 11:28:31,560 epoch 1 - iter 2/2 - loss 3.37699598 - time (sec): 1.88 - samples/sec: 9065.58 - lr: 0.100000
2023-10-11 11:28:31,562 ----------------------------------------------------------------------------------------------------
2023-10-11 11:28:31,563 EPOCH 1 done: loss 3.3770 - lr 0.100000


100%|██████████| 1/1 [00:00<00:00,  1.27it/s]

2023-10-11 11:28:32,363 Evaluating as a multi-label problem: False
2023-10-11 11:28:32,381 DEV : loss 2.057619094848633 - f1-score (micro avg)  0.0
2023-10-11 11:28:32,389 BAD EPOCHS (no improvement): 0
2023-10-11 11:28:32,390 saving best model


...

2023-10-11 11:34:47,698 Evaluating as a multi-label problem: False
2023-10-11 11:34:47,716 DEV : loss 0.21553461253643036 - f1-score (micro avg)  0.6146
2023-10-11 11:34:47,724 Epoch   118: reducing learning rate of group 0 to 9.7656e-05.
2023-10-11 11:34:47,725 BAD EPOCHS (no improvement): 4
2023-10-11 11:34:47,727 ----------------------------------------------------------------------------------------------------
2023-10-11 11:34:47,728 ----------------------------------------------------------------------------------------------------
2023-10-11 11:34:47,730 learning rate too small - quitting training!
2023-10-11 11:34:47,732 ----------------------------------------------------------------------------------------------------


2023-10-11 11:34:48,480 ----------------------------------------------------------------------------------------------------
2023-10-11 11:34:49,003 SequenceTagger predicts: Dictionary with 23 tags: O, S-WINNER, B-WINNER, E-WINNER, I-WINNER, S-DATE, B-DATE, E-DATE, I-DATE, S-NATIONALITY, B-NATIONALITY, E-NATIONALITY, I-NATIONALITY, S-PRIZETYPE, B-PRIZETYPE, E-PRIZETYPE, I-PRIZETYPE, S-REASON, B-REASON, E-REASON, I-REASON, <START>, <STOP>


100%|██████████| 1/1 [00:00<00:00,  2.18it/s]

2023-10-11 11:34:49,548 Evaluating as a multi-label problem: False
2023-10-11 11:34:49,562 0.6489	0.4106	0.503	0.336
2023-10-11 11:34:49,564 
Results:
- F-score (micro) 0.503
- F-score (macro) 0.4216
- Accuracy 0.336

By class:
              precision    recall  f1-score   support

      WINNER     0.5200    0.2955    0.3768        88
        DATE     0.7736    0.6308    0.6949        65
 NATIONALITY     0.7059    0.3158    0.4364        38
   PRIZETYPE     0.5455    0.6667    0.6000         9
      REASON     0.0000    0.0000    0.0000         7

   micro avg     0.6489    0.4106    0.5030       207
   macro avg     0.5090    0.3817    0.4216       207
weighted avg     0.6173    0.4106    0.4846       207

2023-10-11 11:34:49,566 ----------------------------------------------------------------------------------------------------


{'test_score': 0.5029585798816568,
 'dev_score_history': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.021052631578947368,
  0.021352313167259787,
  0.0410958904109589,
  0.03472222222222222,
  0.11147540983606558,
  0.0352112676056338,
  0.03745318352059925,
  0.15333333333333335,
  0.036101083032490974,
  0.0704225352112676,
  0.0586080586080586,
  0.14864864864864866,
  0.1901639344262295,
  0.20846905537459282,
  0.16611295681063123,
  0.3657142857142857,
  0.25,
  0.3573487031700288,
  0.46875,
  0.49104859335038364,
  0.5270935960591132,
  0.5051020408163265,
  0.5339805825242719,
  0.5257985257985257,
  0.5239294710327456,
  0.5454545454545454,
  0.558252427184466,
  0.5413533834586466,
  0.5202020202020202,
  0.5289672544080605,
  0.56730

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

# load the trained model
model = SequenceTagger.load("resources/taggers/ner/final-model.pt")

# create example sentence
sentence = Sentence('Barack Hussein Obama II ( born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African-American president. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004, and worked as a civil rights lawyer and university lecturer. ')

# predict the tags
model.predict(sentence)
print(sentence.to_tagged_string())

2023-10-11 11:45:49,348 SequenceTagger predicts: Dictionary with 23 tags: O, S-WINNER, B-WINNER, E-WINNER, I-WINNER, S-DATE, B-DATE, E-DATE, I-DATE, S-NATIONALITY, B-NATIONALITY, E-NATIONALITY, I-NATIONALITY, S-PRIZETYPE, B-PRIZETYPE, E-PRIZETYPE, I-PRIZETYPE, S-REASON, B-REASON, E-REASON, I-REASON, <START>, <STOP>
Sentence[79]: "Barack Hussein Obama II ( born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African-American president. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004, and worked as a civil rights lawyer and university lecturer." → ["August 4, 1961"/DATE, "American"/NATIONALITY, "he"/WINNER, "U.S."/NATIONALITY]


In [ ]:
from flair.embeddings import TransformerWordEmbeddings
from flair.models import RelationExtractor

relation_label_dict = corpus.make_label_dictionary(label_type="relation")
embeddings = TransformerWordEmbeddings()
results_base_path = 'resources/relations/'
model: RelationExtractor = RelationExtractor(
        embeddings=embeddings,
        label_dictionary=relation_label_dict,
        label_type="relation",
        entity_label_type="ner",
        train_on_gold_pairs_only=True,
    )

trainer: ModelTrainer = ModelTrainer(model, corpus)

trainer.train(
        results_base_path,
        learning_rate=0.1,
        mini_batch_size=2,
        max_epochs=3,
        shuffle=False,
    )

2023-10-11 11:49:56,894 Computing label dictionary. Progress:


57it [00:00, 18025.74it/s]

2023-10-11 11:49:56,904 ERROR: You specified label_type='relation' which is not in this dataset!
2023-10-11 11:49:56,905 ERROR: The corpus contains the following label types: 'ner' (in 57 sentences)


Exception: ignored

In [ ]:
import json
file = {}
with open('annotated.json') as f:
    file = json.load(f)
    # print(file[1]['annotations'])



In [ ]:
from jinja2.nodes import If
import re

position = 0
next_label = 0
labels = [item for item in file[0]['annotations'][0]['result'] if item['type'] == 'labels']
item_count = 1
ids = {} # count -> id
# Start of a document, so we write the relations down
# relations = 7;7;1;2;founded_by|7;7;4;5;founded_by
for item in [item for item in re.split(r'(\s+)|([\(,\.\)\\;:$*&"-])|(\'\w+)|(\')', file[0]['data']['text']) if item]:
  if re.search(r'\s+', item):
    position += len(item)
    continue
  start = position
  end = position + len(item)
  row = f'{item_count}\t{item}'
  label = '\tO'
  if start >= labels[next_label]['value']['start'] and end <= labels[next_label]['value']['end']:
    # the current item has a label, and might have a relation
    id = labels[next_label]["id"]
    ids[item_count] = id
    if start == labels[next_label]['value']['start']:
      # Current Item is start of a label
      label = '\tB-' + labels[next_label]["value"]["labels"][0]
    else:
      # Current item is inside a label
      label = '\tI-' + labels[next_label]["value"]["labels"][0]

    if end >= labels[next_label]['value']['end']:
      next_label += 1

  # relations = [item for item in file[0]['annotations'][0]['result'] if item['type'] == 'relation']
  row = row + label
  position += len(item)
  item_count += 1


# print(re.find(r'(\s+|([\(,\.\)\\])|(\'\w+)|(\')', file[0]['data']['text']))


In [ ]:
line = '# relations = '
keys = ids.keys()
relations = [item for item in file[0]['annotations'][0]['result'] if item['type'] == 'relation']
print(relations)
for relation in relations:
  from_ids = []
  to_ids = []
  rel_type = relation['labels'][0]
  for key in keys:
    if ids[key] == relation['from_id']:
      from_ids.append(key)
    if ids[key] == relation['to_id']:
      to_ids.append(key)
  from_ids = [int(i) for i in from_ids].sorted()
  to_ids = [int(i) for i in to_ids].sorted()
  f'{from_ids[0]};{from_ids[-1]};{to_ids[0]};{to_ids[-1]};{rel_type}'


[{'type': 'relation', 'to_id': 'D_w-bbv0FW', 'labels': ['born_on'], 'from_id': 'qOz6CXctki', 'direction': 'right'}, {'type': 'relation', 'to_id': '6IEOyiTTRR', 'labels': ['died_on'], 'from_id': 'qOz6CXctki', 'direction': 'right'}, {'type': 'relation', 'to_id': '5SuvyzVQn4', 'labels': ['has_nationality'], 'from_id': 'qOz6CXctki', 'direction': 'right'}, {'type': 'relation', 'to_id': 'eoEtlIY3dq', 'labels': ['has_nationality'], 'from_id': 'WASMtd3xiq', 'direction': 'right'}, {'type': 'relation', 'to_id': '50aILF-wsY', 'labels': ['received_nobelprize_in'], 'from_id': 'lmkQq4uhNr', 'direction': 'right'}, {'type': 'relation', 'to_id': 'fDBIqiWAQ2', 'labels': ['has_won'], 'from_id': 'lmkQq4uhNr', 'direction': 'right'}, {'type': 'relation', 'to_id': 'Ytc4bT8tuJ', 'labels': ['has_nationality'], 'from_id': 'WASMtd3xiq', 'direction': 'right'}]


In [ ]:
file[0]['annotations'][0]['result']
# labels = [item for item in file[0]['annotations'][0]['result'] if item['type'] == 'labels']
relations = [item for item in file[0]['annotations'][0]['result'] if item['type'] == 'relation']
print(relations)

[{'type': 'relation', 'to_id': 'D_w-bbv0FW', 'labels': ['born_on'], 'from_id': 'qOz6CXctki', 'direction': 'right'}, {'type': 'relation', 'to_id': '6IEOyiTTRR', 'labels': ['died_on'], 'from_id': 'qOz6CXctki', 'direction': 'right'}, {'type': 'relation', 'to_id': '5SuvyzVQn4', 'labels': ['has_nationality'], 'from_id': 'qOz6CXctki', 'direction': 'right'}, {'type': 'relation', 'to_id': 'eoEtlIY3dq', 'labels': ['has_nationality'], 'from_id': 'WASMtd3xiq', 'direction': 'right'}, {'type': 'relation', 'to_id': '50aILF-wsY', 'labels': ['received_nobelprize_in'], 'from_id': 'lmkQq4uhNr', 'direction': 'right'}, {'type': 'relation', 'to_id': 'fDBIqiWAQ2', 'labels': ['has_won'], 'from_id': 'lmkQq4uhNr', 'direction': 'right'}, {'type': 'relation', 'to_id': 'Ytc4bT8tuJ', 'labels': ['has_nationality'], 'from_id': 'WASMtd3xiq', 'direction': 'right'}]


In [ ]:
loaded_model: RelationExtractor = RelationExtractor.load(results_base_path / "final-model.pt")
loaded_model.train_on_gold_pairs_only = False

sentence = Sentence(["Apple", "was", "founded", "by", "Steve", "Jobs", "."])
sentence[0:1].add_label("ner", "ORG")
sentence[4:6].add_label("ner", "PER")

loaded_model.predict(sentence)

sentence.get_labels("relation")[0].value

In [ ]:
labels[next_label]

{'id': 'qOz6CXctki',
 'type': 'labels',
 'value': {'end': 12,
  'text': 'Albert Camus',
  'start': 0,
  'labels': ['Winner']},
 'origin': 'manual',
 'to_name': 'text',
 'from_name': 'label'}